In [23]:
import numpy as np
import pandas as pd
from pprint import pprint

We import the information from the Excel file

In [51]:
table = pd.read_excel('phones.xlsx')
table.index = table['ID']
table = table.drop('ID', axis=1)

In [52]:
table

,Price (CAD),Processor Speed (GHz),Capacity (GB),Rear camera resolution (MP),Battery capacity (mAh)
ID,,,,,
iPhone XS 64,1379.00,2.40,64,12.0,2658
iPhone XS 256,1589.00,2.40,256,12.0,2658
iPhone XS 512,1859.00,2.40,512,12.0,2658
Galaxy Note 9 128,900.00,2.70,128,12.0,4000
Galaxy note 9 512,1449.00,2.70,512,12.0,4000
Galaxy Note 8,750.00,2.35,64,12.0,3300
Pixel 3 64,800.00,2.50,64,12.2,2915
Pixel 3 128,1215.00,2.50,128,12.2,2915
Moto Z3,470.00,2.35,64,12.0,3000


### We create a dictionnary to easily map from the criteria to their respective weights.

In [128]:
criteria = table.columns
weights  = [0.3, 0.1, 0.2, 0.15, 0.25]

if len(criteria) == len(weights) and np.isclose(1, sum(weight for weight in weights)):
    w_criteria = {criterion:weight for criterion, weight in zip(criteria, weights)}
else:
    print(f'Number of criteria: {len(criteria)}, number of weights: {len(weights)}')
    print(f'Sum of weights: {sum(weight for weight in weights)}')
    w_criteria = {criterion:0 for criterion in criteria}
    raise Exception(f'A weight is needed for each criterion and the sum of weights must be equal to one!')

w_criteria

{'Price (CAD)': 0.3,
 'Processor Speed (GHz)': 0.1,
 'Capacity (GB)': 0.2,
 'Rear camera resolution (MP)': 0.15,
 'Battery capacity (mAh)': 0.25}

### We create a dictionary to access the optimization direction (min or max) for each criterion

In [131]:
senses = [0, 1, 1, 1, 1] # O and 1 because they automatically map to complementary bool values. 

if len(senses) == len(criteria):
    s_criteria = {criterion:sense for criterion, sense in zip(criteria, senses)}
else:
    raise Exception(f'Specify a value (0 for min, 1 for max) for each one of the criteria : {list(criteria)}')

## Create the normalised decision matrix

For the normalisation part, there are many possible rules. The following options are available on this implementation: 
$$ x_{ij} \;\; = \;\; \frac{a_{ij}}{\sqrt{\sum_{i}^{N} a_{ij}}}$$

In [62]:
n_table = table.copy()

sq_sum_squares = table.apply(lambda y: np.sqrt(sum(x**2 for x in y)))
sq_sum_squares = dict(sq_sum_squares)

for column in table.columns:
    f = (lambda y: lambda x: x/sq_sum_squares[y])(column) 
    n_table[column] = table[column].map(f)
    
n_table.head(5)

,Price (CAD),Processor Speed (GHz),Capacity (GB),Rear camera resolution (MP),Battery capacity (mAh)
ID,,,,,
iPhone XS 64,0.303162,0.268521,0.063977,0.254704,0.213125
iPhone XS 256,0.349329,0.268521,0.255910,0.254704,0.213125
iPhone XS 512,0.408686,0.268521,0.511820,0.254704,0.213125
Galaxy Note 9 128,0.197858,0.302086,0.127955,0.254704,0.320730
Galaxy note 9 512,0.318551,0.302086,0.511820,0.254704,0.320730


# EXPLORE MORE RULES !

### Create the weighted normalised decision matrix

In [63]:
w_n_table = n_table.copy()

for column in n_table.columns: 
    w_n_table[column] = n_table[column].map(lambda x: x*w_criteria[column])
    
w_n_table.head(5)

,Price (CAD),Processor Speed (GHz),Capacity (GB),Rear camera resolution (MP),Battery capacity (mAh)
ID,,,,,
iPhone XS 64,0.090949,0.026852,0.012795,0.038206,0.053281
iPhone XS 256,0.104799,0.026852,0.051182,0.038206,0.053281
iPhone XS 512,0.122606,0.026852,0.102364,0.038206,0.053281
Galaxy Note 9 128,0.059357,0.030209,0.025591,0.038206,0.080183
Galaxy note 9 512,0.095565,0.030209,0.102364,0.038206,0.080183


### Computation of the concordance matrix

In [93]:
concordance_matrix = pd.DataFrame(columns=table.index, index=table.index)

for phone in w_n_table.index:
    for phone2 in w_n_table.index:
        _sum = 0
        for criterion in w_n_table.columns:
            if w_n_table.loc[phone, criterion] >= w_n_table.loc[phone2, criterion]:
                _sum += w_criteria[criterion]
        if phone == phone2:
            concordance_matrix.loc[phone, phone2] = 0
        else:
            concordance_matrix.loc[phone, phone2] = _sum
        
concordance_matrix

ID,iPhone XS 64,iPhone XS 256,iPhone XS 512,Galaxy Note 9 128,Galaxy note 9 512,Galaxy Note 8,Pixel 3 64,Pixel 3 128,Moto Z3,Galaxy s10e 128,Galaxy s10e 256,Galaxy s10 128,Galaxy s10 512,Galaxy M10 16,Galaxy M10 32
ID,,,,,,,,,,,,,,,
iPhone XS 64,0,0.5,0.5,0.45,0.15,0.75,0.5,0.3,0.75,0.55,0.55,0.25,0.25,0.6,0.6
iPhone XS 256,1,0,0.5,0.65,0.45,0.75,0.5,0.5,0.75,0.75,0.75,0.75,0.25,0.6,0.6
iPhone XS 512,1,1,0,0.65,0.65,0.75,0.5,0.5,0.75,0.75,0.75,0.75,0.75,0.6,0.6
Galaxy Note 9 128,0.7,0.5,0.5,0,0.5,1,0.85,0.55,1,0.7,0.5,0.7,0.5,0.85,0.85
Galaxy note 9 512,1,0.7,0.7,1,0,1,0.85,0.85,1,1,1,1,0.7,0.85,0.85
Galaxy Note 8,0.6,0.4,0.4,0.15,0.15,0,0.45,0.25,1,0.5,0.5,0.25,0.25,0.6,0.6
Pixel 3 64,0.7,0.5,0.5,0.15,0.15,0.75,0,0.5,0.75,0.25,0.25,0.25,0.25,0.6,0.6
Pixel 3 128,0.7,0.5,0.5,0.65,0.15,0.75,1,0,0.75,0.75,0.55,0.45,0.25,0.6,0.6
Moto Z3,0.6,0.4,0.4,0.15,0.15,0.45,0.45,0.25,0,0.25,0.25,0.25,0.25,0.6,0.6


### Binary concordance set

In [106]:
binary_concordance_matrix = concordance_matrix.copy()

sum_of_sums_of_columns = sum(concordance_matrix[column].sum() for column in concordance_matrix.columns)
non_diagonal_entries = concordance_matrix.shape[0]**2 - concordance_matrix.shape[0]
c_bar = sum_of_sums_of_columns / non_diagonal_entries

binary_concordance_matrix = concordance_matrix.applymap(lambda x: 1 if x > c_bar else 0)

binary_concordance_matrix

ID,iPhone XS 64,iPhone XS 256,iPhone XS 512,Galaxy Note 9 128,Galaxy note 9 512,Galaxy Note 8,Pixel 3 64,Pixel 3 128,Moto Z3,Galaxy s10e 128,Galaxy s10e 256,Galaxy s10 128,Galaxy s10 512,Galaxy M10 16,Galaxy M10 32
ID,,,,,,,,,,,,,,,
iPhone XS 64,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1
iPhone XS 256,1,0,0,1,0,1,0,0,1,1,1,1,0,1,1
iPhone XS 512,1,1,0,1,1,1,0,0,1,1,1,1,1,1,1
Galaxy Note 9 128,1,0,0,0,0,1,1,0,1,1,0,1,0,1,1
Galaxy note 9 512,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1
Galaxy Note 8,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1
Pixel 3 64,1,0,0,0,0,1,0,0,1,0,0,0,0,1,1
Pixel 3 128,1,0,0,1,0,1,1,0,1,1,0,0,0,1,1
Moto Z3,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1


In [ ]:
negatives = lambda y: list(filter(lambda x: True if x < 0 else False, y))
positives = lambda y: list(filter(lambda x: True if x > 0 else False, y))

[-4]